# Auto Color Adjustment
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [auto_color_adjustment.ipynb](https://github.com/diegoinacio/computer-vision-notebooks/blob/master/Computer-Vision-Fundamentals/auto_color_adjustment.ipynb)
---
Methods which provide color adjustment without input parameters.

With these techniques, all result is given by the auto analysis from the input image.

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import imageio

from _utils import *

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Input RGB image
---

In [ ]:
img_in = imageio.imread('../_data/aerial01.png')/255\

In [ ]:
histogram(img_in, bins=2**8, interval=[0, 1])

In [ ]:
summary(img_in)

## Equalize (Automatic contrast enhancement)
---
Automatic contrast enhancement using *histogram equalization* for each channel.

In [ ]:
img_eq = img_in.copy()
# Split channels
R = img_eq[:,:,0]
G = img_eq[:,:,1]
B = img_eq[:,:,2]
# Get dimensions
N1, N2, _ = img_eq.shape
# Histogram equalization
Rs = np.sort(R.ravel())
Gs = np.sort(G.ravel())
Bs = np.sort(B.ravel())
R[:,:] = Rs.searchsorted(R)
G[:,:] = Gs.searchsorted(G)
B[:,:] = Bs.searchsorted(B)
# Feature normalization
img_eq = img_eq/(N1*N2 - 1)

In [ ]:
histogram(img_eq, bins=2**8, interval=[0, 1])

In [ ]:
summary(img_eq)

## RGB component stretch
---
Feature scaling, using *min-max normalization* to stretch each channel to 0.0-1.0 range.

In [ ]:
img_rgb = img_in.copy()
# Get minimum values
RGB_min = np.apply_over_axes(
    np.amin, img_rgb, [0, 1]
).ravel()
# Get maximum values
RGB_max = np.apply_over_axes(
    np.amax, img_rgb, [0, 1]
).ravel()
# min-max normalization
img_rgb = (img_rgb - RGB_min)/(RGB_max - RGB_min)

In [ ]:
histogram(img_rgb, bins=2**8, interval=[0, 1])

In [ ]:
summary(img_rgb)

## HSV component stretch
---
Stretch components from HSV space to 0.0-1.0 range, only preserving the *hue* channel.

In [ ]:
# Convert from RGB to HSV
HSV_ = mpl.colors.rgb_to_hsv(img_in)
# Get minimum values from HSV
HSV_min = np.apply_over_axes(
    np.amin, HSV_, [0, 1]
).ravel()
# Get maximum values from HSV
HSV_max = np.apply_over_axes(
    np.amax, HSV_, [0, 1]
).ravel()
# Find delta = max - min
HSV_delta = HSV_max - HSV_min
# Preserve hue component
HSV_min[[0]] = 0
HSV_delta[0] = 1
# Stretch the components, preserving hue
HSV_ = (HSV_ - HSV_min)/HSV_delta
# Convert from HSV to RGB
img_hsv = mpl.colors.hsv_to_rgb(HSV_)

In [ ]:
histogram(img_hsv, bins=2**8, interval=[0, 1])

In [ ]:
summary(img_hsv)

## White balance
---
White balance based on the concept of *gray world*, where $\mu_R=\mu_G=\mu_B$.

In [ ]:
# Find the mean values
R_mu, G_mu, B_mu = np.apply_over_axes(
    np.mean, img_in, [0, 1]
).ravel()
# Define the correction values based on green channel
corrR = G_mu - R_mu
corrB = G_mu - B_mu
# Offset the components to equalize all mean values
offset = img_in + [corrR, 0 ,corrB]
# Move the mean value to zero
cent = offset - G_mu
# Find the maximum values after offset
cent_max = np.apply_over_axes(
    np.amax, cent, [0, 1]
).ravel()
# Stretch and normalize values gratter than mean
img_wb = (1 - G_mu)*cent/cent_max + G_mu
# Stretch and normalize values lower than mean
cent = offset + 1 - G_mu
cent_min = np.apply_over_axes(
    np.amin, cent, [0, 1]
).ravel()
norm = 1 - (G_mu*(1 - cent)/(1 - cent_min) + 1 - G_mu)
mu_down = norm < G_mu
img_wb[mu_down] = norm[mu_down]

In [ ]:
histogram(img_wb, bins=2**8, interval=[0, 1])

In [ ]:
summary(img_wb)

## Comparison
---

In [ ]:
print('Input')
histogram(img_in, bins=2**8, interval=[0, 1])
print('Equalize')
histogram(img_eq, bins=2**8, interval=[0, 1])
print('RGB component stretch')
histogram(img_rgb, bins=2**8, interval=[0, 1])
print('HSV component stretch')
histogram(img_hsv, bins=2**8, interval=[0, 1])
print('White balance')
histogram(img_wb, bins=2**8, interval=[0, 1])